In [ ]:
"""
Created on Thu Feb 14 20:48:27 2019

@author: CoeFamily
"""

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 15 13:17:09 2019

@author: CoeFamily
"""

In [ ]:
from awips.dataaccess import DataAccessLayer
from dynamicserialize.dstypes.com.raytheon.uf.common.time import TimeRange
import numpy as np
import cartopy.crs as ccrs
import warnings
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
from shapely.geometry import Polygon
from metpy.plots import StationPlot
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pytz
from datetime import datetime, timedelta

In [ ]:
def is_dst(zonename):
    tz = pytz.timezone(zonename)
    now = pytz.utc.localize(datetime.utcnow())
    return now.astimezone(tz).dst() != timedelta(0)

In [ ]:
edexServer = "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost(edexServer)
# EDEX request for multiple states
request = DataAccessLayer.newDataRequest('maps')
request.addIdentifier('table', 'mapdata.states')
request.addIdentifier('geomField', 'the_geom')
request.addIdentifier('inLocation', 'true')
request.addIdentifier('locationField', 'state')
request.setParameters('state','name','lat','lon')
request.setLocationNames('MA','CT','NH','ME','VT','RI','NY',)
response = DataAccessLayer.getGeometryData(request)
print("Found " + str(len(response)) + " MultiPolygons")

In [ ]:
# Append each geometry to a numpy array
states = np.array([])
for ob in response:
    print(ob.getString('name'), ob.getString('state'), ob.getNumber('lat'), ob.getNumber('lon'))
    states = np.append(states,ob.getGeometry())

In [ ]:
# New obs request
edexServer = "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost(edexServer)
request = DataAccessLayer.newDataRequest('maps')
request.addIdentifier('table', 'mapdata.states')
request.addIdentifier('state', 'MA')
request.addIdentifier('geomField', 'the_geom')
request.setParameters('state','name','lat','lon')
response = DataAccessLayer.getGeometryData(request)
record = response[0]
print("Found " + str(len(response)) + " MultiPolygon")
state={}
state['name'] = record.getString('name')
state['state'] = record.getString('state')
state['lat'] = record.getNumber('lat')
state['lon'] = record.getNumber('lon')
#state['geom'] = record.getGeometry()
state['bounds'] = record.getGeometry().bounds

In [ ]:
def make_map(bbox, proj=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(16,12),subplot_kw=dict(projection=proj))
    ax.set_extent(bbox)
    gl = ax.gridlines(draw_labels=True, color='#e7e7e7')
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
# buffer our bounds by +/i degrees lat/lon
bounds = state['bounds']
bbox=[bounds[0]-0.5,bounds[2]+0.5,bounds[1],bounds[3]+1.5]

In [ ]:
fig, ax = make_map(bbox=bbox)
shape_feature = ShapelyFeature(states,ccrs.PlateCarree(),
                    facecolor='none', linestyle="-",edgecolor='#000000',linewidth=2)
ax.add_feature(shape_feature)

In [ ]:
# Create envelope geometry
envelope = Polygon([(bbox[0],bbox[2]),(bbox[0],bbox[3]),
                    (bbox[1], bbox[3]),(bbox[1],bbox[2]),
                    (bbox[0],bbox[2])])

In [ ]:
# New obs request
DataAccessLayer.changeEDEXHost(edexServer)
request = DataAccessLayer.newDataRequest("obs", envelope=envelope)
availableProducts = DataAccessLayer.getAvailableParameters(request)
single_value_params = ["timeObs", "stationName", "longitude", "latitude",
                       "temperature", "dewpoint", "windDir",
                       "windSpeed", "seaLevelPress"]
multi_value_params = ["presWeather", "skyCover", "skyLayerBase"]
params = single_value_params + multi_value_params
request.setParameters(*(params))

In [ ]:
# Time range
lastHourDateTime = datetime.utcnow() - timedelta(minutes = 60)
start = lastHourDateTime.strftime('%Y-%m-%d %H:%M:%S')
end = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
beginRange = datetime.strptime( start , "%Y-%m-%d %H:%M:%S")
endRange = datetime.strptime( end , "%Y-%m-%d %H:%M:%S")
timerange = TimeRange(beginRange, endRange)
# Get response
response = DataAccessLayer.getGeometryData(request,timerange)
# function getMetarObs was added in python-awips 18.1.4
obs = DataAccessLayer.getMetarObs(response)
print("Found " + str(len(response)) + " records")
print("Using " + str(len(obs['temperature'])) + " temperature records")
# Create a station plot pointing to an Axes to draw on as well as the location of points
now = datetime.now()
hours = now.hour
if(hours > 12):
    hours = hours - 12
    ampm = "PM"
elif(hours == 12):
    ampm = "PM"
else:
    ampm = "AM"

In [ ]:
timee = str(now.month) + "-" + str(now.day) + "-" + str(now.year) + " " + str(hours) + ":" + str(now.minute) + " " + ampm
ax.set_title(timee + " | Current Surface Temps in Degrees F", fontsize=20)

In [ ]:
lats = obs['latitude']
lons = obs['longitude']

In [ ]:
thresholds = {
    '15': 'purple',
    '25': 'c',
    '35': 'royalblue',
    '45': 'darkgreen', 
    '55': 'green',
    '65': 'y', 
    '75': 'orange',
    '85': 'red'
}

In [ ]:
tempp = np.array(obs['temperature'], dtype=float)

In [ ]:
if(len(tempp) > 14):

	# Suppress nan masking warnings
	warnings.filterwarnings("ignore",category =RuntimeWarning)

	for x, value in thresholds.items():
    		tair = np.array(obs['temperature'], dtype=float)
    		tair[tair == -9999.0] = 'nan'
    		tair = (tair*1.8)+32
    		if x==max(thresholds):
       		        tair[(tair < int(x))] = 'nan'
    		elif x==min(thresholds):
        		tair[(tair >= int(x)+10)] = 'nan'
    		else:
        		tair[(tair < int(x))] = 'nan'
        		tair[(tair >= int(x)+10)] = 'nan'

    		stationplot = StationPlot(ax, lons, lats, transform=ccrs.PlateCarree(),
                          		fontsize=14)
    		stationplot.plot_parameter('C', tair, color=thresholds[str(x)])

	plt.savefig('temp.png', bbox_inches='tight', pad_inches= 0)